In [17]:
import numpy as np

class SlotMachine:
    def __init__(self, height, width, reels, reel_length, payoff, free_spins, seed = 2022):
        self.height = height
        self.width = width
        self.window = np.zeros((height, width))
        self.reels = reels
        self.reel_length = reel_length
        self.payoff = payoff
        self.wild = 2
        self.scatter = 1
        self.free_spins = free_spins
        self.bonus_play = 0
        self.rng = np.random.default_rng(seed)

    def spin(self):
        stops = self.rng.integers(0, self.reel_length, size=self.width, dtype=np.int32, endpoint=False)
        for i in range(self.width):
            for j in range(self.height):
                self.window[j][i] = self.reels[i][(stops[i] + j) % self.reel_length]

    def pay(self):
        payment = 0
        for i in range(self.height):
            pay_symbol = self.window[i][0]
            winlines = []
            for j in range(self.width):
                reel = self.window[:, j]
                y = sum([np.count_nonzero(reel == x) for x in [pay_symbol, self.wild]])
                if y > 0:
                    winlines.append(y)
                else:
                    break
            payment += self.payoff[int(pay_symbol)][len(winlines) - 1] * np.prod(winlines)
            if len(winlines) >= 3:
                self.probs[int(pay_symbol) - 1][len(winlines) - 3] +=  np.prod(winlines)
        return payment

    def get_free_spins(self):
        c = 0
        for i in range(self.width):
            reel = self.window[:, i]
            c += np.count_nonzero(reel == self.scatter)
        if c >= 3:
            self.spins_with_bonus_spins += 1
        for k, v in self.free_spins.items():
            self.bonus_play += int(c / k) * v
            c = c % k

    def play(self, iters):
        payment = 0
        total_iters = 0;
        self.probs = np.zeros((11, 3))
        self.bonus_play = 0
        total_bonus_spin = 0
        self.spins_with_bonus_spins = 0;
        for i in range(iters):
            self.spin()
            total_iters += 1
            payment += self.pay()
            self.get_free_spins()
            while self.bonus_play > 0:
                self.bonus_play -= 1
                total_bonus_spin += 1
                self.spin()
                total_iters += 1
                payment += self.pay()
                self.get_free_spins()
        return {'RTP' : payment * 100.0 / (iters * 1000),
                'probs': self.probs[2:, :] / total_iters,
                'mean value of free spins' : total_bonus_spin/iters,
                'probability of a free spin' : self.spins_with_bonus_spins / total_iters}

In [19]:
reel_1 = [10, 1, 4, 5, 8, 11, 4, 5, 9, 8, 7, 11, 10, 6, 7, 9, 8, 10, 3, 9, 11, 6, 9]
reel_2345 = [6, 1, 3, 8, 7, 6, 9, 11, 2, 8, 4, 5, 10, 11, 4, 10, 8, 7, 9, 11, 5, 10, 9]
payoff = [
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 2000, 10000, 20000],
    [0, 0, 1000, 5000, 10000],
    [0, 0, 1000, 4000, 6000],
    [0, 0, 1000, 2000, 5000],
    [0, 0, 500, 1000, 4000],
    [0, 0, 500, 1000, 4000],
    [0, 0, 100, 500, 2000],
    [0, 0, 100, 500, 2000],
    [0, 0, 100, 500, 2000]
]
free_spins = {5: 10, 4: 8, 3: 6}
reels = [reel_1, reel_2345, reel_2345, reel_2345, reel_2345]
reel_length = len(reel_1)
sm = SlotMachine(height=3, width=5, reels=reels, reel_length=reel_length, payoff=payoff, free_spins=free_spins)
print(sm.play(100000))
rtps = []
n = 10
for i in range(n):
    sm = SlotMachine(height=3, width=5, reels=reels, reel_length=reel_length, payoff=payoff, free_spins=free_spins, seed = i)
    rtps.append(sm.play(100000)['RTP'])
print('RTPs: ', rtps)
print('mean RTP', np.mean(rtps))

{'RTP': 93.7329, 'probs': array([[0.00667176, 0.00159021, 0.00067517],
       [0.02560321, 0.00970115, 0.00599659],
       [0.02419957, 0.00915923, 0.00608543],
       [0.02424399, 0.00969226, 0.00613873],
       [0.02442166, 0.00916812, 0.00609431],
       [0.05977044, 0.03142212, 0.03018727],
       [0.07416225, 0.03772965, 0.0384137 ],
       [0.05177499, 0.02563875, 0.02845492],
       [0.05999254, 0.03013397, 0.02865925]]), 'mean value of free spins': 0.12564, 'probability of a free spin': 0.018140791016666075}
RTPs:  [94.2692, 93.4655, 93.7613, 93.6353, 95.0368, 93.8778, 94.9938, 93.524, 92.6021, 94.3805]
mean RTP 93.95463
